## Predicting Customer Churns

In [153]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [154]:
churn = pd.read_csv('online_retail.csv')
churn.head()

,Transaction_ID,Product_Code,Product_Title,Units_Sold,Transaction_Timestamp,Unit_Cost,Client_ID,Client_Region
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,01-12-2009 07:45,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,01-12-2009 07:45,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,01-12-2009 07:45,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,01-12-2009 07:45,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,01-12-2009 07:45,1.25,13085.0,United Kingdom


In [155]:
# Dataframe Shape
churn.shape

(525461, 8)

In [156]:
# Finding the number of missing values in each column
churn.isna().sum()

Transaction_ID                0
Product_Code                  0
Product_Title              2928
Units_Sold                    0
Transaction_Timestamp         0
Unit_Cost                     0
Client_ID                107927
Client_Region                 0
dtype: int64

## Data Preprocessing

In [157]:
# Dropping duplicate rows
churn.drop_duplicates(inplace = True)

In [158]:
churn.shape

(518596, 8)

In [159]:
print("Unique product codes:", churn["Product_Code"].nunique())
print("Unique product titles:", churn["Product_Title"].nunique())
print("Missing product titles:", churn["Product_Title"].isnull().sum())

Unique product codes: 4632
Unique product titles: 4681
Missing product titles: 2928


In [160]:
# There are more product codes than product titles

In [161]:
# How many product codes have more than one unique title
more_titles = churn[churn["Product_Title"].notnull()].groupby("Product_Code")["Product_Title"].nunique()
print("Product codes with more than 1 titles:", ( more_titles > 1).sum())

Product codes with more than 1 titles: 687


In [162]:
conflict_codes = more_titles[more_titles > 1].index

In [163]:
churn[churn["Product_Code"].isin(conflict_codes)][['Product_Code', 'Product_Title']].drop_duplicates().sort_values("Product_Code").head(20)

,Product_Code,Product_Title
2944,10120,DOGGY RUBBER
194372,10120,Zebra invcing error
614,15056N,EDWARDIAN PARASOL NATURAL
100429,15056N,wedding co returns?
54894,15058B,PINK WHITE SPOTS GARDEN PARASOL
461044,15058B,PINK POLKADOT GARDEN PARASOL
10462,16011,ANIMAL STICKERS
119710,16011,ANIMAL STICKERS
4157,16012,FOOD/DRINK SPUNGE STICKERS
314800,16012,FOOD/DRINK SPONGE STICKERS


In [164]:
# There are some product titles that are irregular and junk
# we need to remove them
junk_keywords = ["found", "missing", "damages", "error", "returns", "invcing", "nan", "?"]
churn["Title_lower"] = churn["Product_Title"].str.lower()

In [165]:
# creating a pattern to remove junk titles
import re
#junk_pattern = "|".join(junk_keywords)
junk_pattern = "|".join([re.escape(word) for word in junk_keywords])
print(junk_pattern)

found|missing|damages|error|returns|invcing|nan|\?


In [166]:
# removing rows with product names as junk titles
clean_titles = churn[(churn['Product_Title'].notnull()) & (~churn['Title_lower'].str.contains(junk_pattern, na=False))]

In [167]:
# grouping the rows by product code
grouped = clean_titles.groupby("Product_Code")
# we need to assign a single title that is repeated most to each product code
common_titles = {}
for product_code, group in grouped:
    title_counts = group["Product_Title"].value_counts()
    common_title = title_counts.idxmax()
    common_titles[product_code] = common_title
# this is a dictionary with product codes with mostky repeated product titles

In [168]:
# map product codes to product titles from the dictionary
churn['Product_Title_Cleaned'] = churn['Product_Code'].map(common_titles)

In [169]:
churn.drop(columns=['Title_lower', 'Product_Title'], inplace=True)

In [170]:
print(f"Dataframe shape after cleaning product titles: {churn.shape}")

Dataframe shape after cleaning product titles: (518596, 8)


In [171]:
churn.isna().sum()

Transaction_ID                0
Product_Code                  0
Units_Sold                    0
Transaction_Timestamp         0
Unit_Cost                     0
Client_ID                107833
Client_Region                 0
Product_Title_Cleaned       780
dtype: int64

In [172]:
print("Unique product codes:", churn["Product_Code"].nunique())
print("Unique product titles:", churn["Product_Title_Cleaned"].nunique())

Unique product codes: 4632
Unique product titles: 4103
